In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./vaswani2017-AttentionIsAllYouNeed.pdf")
docs = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:30], OllamaEmbeddings(model="komodo7b"))

C:\Users\HP\AppData\Local\Temp\ipykernel_7496\1114713052.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(documents[:30], OllamaEmbeddings(model="komodo7b"))


In [4]:
query = "Apa itu Attention Mechanism"
result = db.similarity_search(query)
print(result[0].page_content)

√dk
)V (1)
The two most commonly used attention functions are additive attention [2], and dot-product (multi-
plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of 1√dk
. Additive attention computes the compatibility function using a feed-forward network with
a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized
matrix multiplication code.
While for small values of dk the two mechanisms perform similarly, additive attention outperforms
dot product attention without scaling for larger values of dk [3]. We suspect that for large values of
dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients 4. To counteract this effect, we scale the dot products by 1√dk
.
3.2.2 Multi-Head Attention


In [5]:
from langchain_community.llms import Ollama

llm = Ollama(model="komodo7b")
llm

C:\Users\HP\AppData\Local\Temp\ipykernel_7496\4005118269.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="komodo7b")


Ollama(model='komodo7b')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Jawablah pertanyaan saya hanya berdasarkan konteks yang saya berikan.
Berfikirlah langkah demi langkah sebelum memberikan jawaban secara detail.
<context> 
{context}
</context>
Question: {input}                                 
""")

In [7]:
# Chain / Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
# retriever
retriever = db.as_retriever()

In [9]:
# retrieval chain
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
result = retrieval_chain.invoke({
    "input": "Apa itu Attention Mechanism"
})

print(result)

{'input': 'Apa itu Attention Mechanism', 'context': [Document(metadata={'source': './vaswani2017-AttentionIsAllYouNeed.pdf', 'page': 3}, page_content='√dk\n)V (1)\nThe two most commonly used attention functions are additive attention [2], and dot-product (multi-\nplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor\nof 1√dk\n. Additive attention computes the compatibility function using a feed-forward network with\na single hidden layer. While the two are similar in theoretical complexity, dot-product attention is\nmuch faster and more space-efficient in practice, since it can be implemented using highly optimized\nmatrix multiplication code.\nWhile for small values of dk the two mechanisms perform similarly, additive attention outperforms\ndot product attention without scaling for larger values of dk [3]. We suspect that for large values of\ndk, the dot products grow large in magnitude, pushing the softmax function into regions where

In [11]:
result = retrieval_chain.invoke({
    "input": "Apa saja komponen-komponen dari scaled dot-product attention?"
})

print(result["answer"])

Dalam sebuah model Transformer, skala perhatian dot produk adalah komponen utama dalam mencapai hasil yang mengesankan. Komponen-komponen dari scaled dot-product attention termasuk:
1. **Query**: Input ke self-attention layer, yaitu output dari sublayer sebelumnya atau embedding layer. Query menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
2. **Key**: Ini adalah hasil dari sublayer sebelumnya atau embedding layer, yang juga bertindak sebagai input ke self-attention layer. Key menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
3. **Value**: Ini adalah hasil dari sublayer sebelumnya atau embedding layer, yaitu input ke self-attention layer. Value menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
4. **Scaling Factor**: Scaling factor adalah konstanta yang dihasilkan oleh skala perhatian dot-produk. Ini membantu untuk memperbesar atau mengecilkan jumlah informasi yang diterima oleh Query, Key, dan Value,

In [13]:
from langchain_core.output_parsers import StrOutputParser
chain_wo_rag = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please response to the user queries"),
        ("user","Question:{question}")
    ]
) | llm | StrOutputParser()

result_without_rag = chain_wo_rag.invoke({"question":"Apa saja komponen-komponen dari scaled dot-product attention?"})

In [14]:
result_with_rag = result["answer"]
print(f"[ANSWER WITH RAG CHAIN]\n{result_with_rag}")
print()
print(f"[ANSWER WITHOUT RAG CHAIN]\n{result_without_rag}")

[ANSWER WITH RAG CHAIN]
Dalam sebuah model Transformer, skala perhatian dot produk adalah komponen utama dalam mencapai hasil yang mengesankan. Komponen-komponen dari scaled dot-product attention termasuk:
1. **Query**: Input ke self-attention layer, yaitu output dari sublayer sebelumnya atau embedding layer. Query menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
2. **Key**: Ini adalah hasil dari sublayer sebelumnya atau embedding layer, yang juga bertindak sebagai input ke self-attention layer. Key menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
3. **Value**: Ini adalah hasil dari sublayer sebelumnya atau embedding layer, yaitu input ke self-attention layer. Value menunjukkan bagaimana model harus mengambil informasi dari pemetaan berulang. 
4. **Scaling Factor**: Scaling factor adalah konstanta yang dihasilkan oleh skala perhatian dot-produk. Ini membantu untuk memperbesar atau mengecilkan jumlah informasi yang diterima ole

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import ArxivRetriever
from langchain_ollama.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("""
Jawablah pertanyaan saya hanya berdasarkan konteks yang saya berikan.
Berfikirlah langkah demi langkah sebelum memberikan jawaban secara detail.
<context> 
{context}
</context>
Pertanyaan: {input}                                 
""")


llm = ChatOllama(model="komodo7b")

def format_docs(docs):
    document = ""
    for doc in docs:
        paper = "\n".join(f"{k}: {v}" for k, v in doc.metadata.items())
        paper = paper + "\n" + doc.page_content + "\n\n"
        document = document + paper        
    return document

arxiv_retriever = ArxivRetriever(load_max_docs=1, get_full_documents=True)

chain = (
    {"context": arxiv_retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
chain.invoke("Siapakah penulis paper dengan judul SparseSwin: Swin Transformer with Sparse Transformer Block")

'</eot_id>Penulis adalah Xingwen Zhu dan Wenqi Gao.'

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import ArxivRetriever
from langchain_ollama.chat_models import ChatOllama
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

llm = ChatOllama(model="komodo7b")

prompt = hub.pull("hwchase17/react")

api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=4096)
tool = ArxivQueryRun(api_wrapper=api_wrapper)
tools = [tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=64)

In [8]:
agent_executor.invoke({"input": "Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv"})



> Entering new AgentExecutor chain...
 Kita bisa mencari informasi lebih lanjut tentang penulis menggunakan sumber daya kami di arxiv.

Action: arxiv
Action Input: SparseSwin: Swin Transformer with Sparse Transformer BlockPublished: 2023-09-11
Title: SparseSwin: Swin Transformer with Sparse Transformer Block
Authors: Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, Novanto Yudistira
Summary: Advancements in computer vision research have put transformer architecture as
the state of the art in computer vision tasks. One of the known drawbacks of
the transformer architecture is the high number of parameters, this can lead to
a more complex and inefficient algorithm. This paper aims to reduce the number
of parameters and in turn, made the transformer more efficient. We present
Sparse Transformer (SparTa) Block, a modified transformer block with an
addition of a sparse token converter that reduces the number of tokens used. We
use the S

{'input': 'Siapakah penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv',
 'output': 'Penulis paper berjudul SparseSwin: Swin Transformer with Sparse Transformer Block pada ArXiv adalah Krisna Pinasthika, Blessius Sheldo Putra Laksono, Riyandi Banovbi Putera Irsal, Syifa Hukma Shabiyya, dan Novanto Yudistira.'}

In [12]:
user_query = """
Gunakan tools arxiv untuk menjawab pertanyaan di bawah ini:

Pada tahun berapa paper berjudul "Attention is All You Need" yang ditulis oleh Vaswani diterbitkan?
"""
print(agent_executor.invoke({"input": user_query})["output"])



> Entering new AgentExecutor chain...
 Saya akan menggunakan arxiv untuk menjawab pertanyaan ini.Invalid Format: Missing 'Action:' after 'Thought: Action: arxiv
Action Input: "Transformers" AND "Attention is All You Need" AND "Vaswani"Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our mo

In [18]:
user_query = """
Gunakanlah tool ArXiv untuk menjawab pertanyaan di bawah ini:

Siapakah penulis publikasi yang berjudul 'MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning'
"""
print(agent_executor.invoke({"input": user_query})["output"])



> Entering new AgentExecutor chain...
 Sebelum melanjutkan dengan jawaban saya, bisakah Anda memberi tahu saya secara khusus nama penulis yang menerbitkan publikasi yang berjudul "MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning"?Invalid Format: Missing 'Action:' after 'Thought: Maaf, sebagai asisten AI, saya tidak memiliki akses ke arXiv.org dan tidak dapat menjalankan hal itu. Bisakah Anda memberi tahu saya secara khusus nama penulis yang menerbitkan publikasi yang berjudul "MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning"?Invalid Format: Missing 'Action:' after 'Thought: Maaf, sebagai asisten AI, saya tidak memiliki akses ke arXiv.org dan tidak dapat menjalankan hal itu. Bisakah Anda memberi tahu saya secara khusus nama penulis yang menerbitkan publikasi yang berjudul "MAMI: Multi-Attentional Mutual-Information for Long Sequence Neuron Captioning"?Invalid Format: Missing 'Action:' after 'Thought: Maaf, sebagai asisten AI

KeyboardInterrupt: 